# 线性系统求解
$Ax = b  $
$$A = c_0A_0+c_1A_1+c_2A_2=\mathbb{I}+0.2X_0Z_1+0.2X_0$$
$$|b\rangle = U|0\rangle = H_0H_1H_2|0\rangle$$


In [80]:
import numpy as np
import matplotlib
import paddle 
from paddle import matmul
from paddle_quantum.circuit import UAnsatz
from paddle_quantum.utils import random_pauli_str_generator, pauli_str_to_matrix, dagger
#参数设置
num_qubit = 3  
num_shots = 10**6 # Number of quantum measurements
tot_qubits = num_qubit+1 # Addition of an ancillary qubit
ancilla_idx = num_qubit #Index of the ancillary qubit (last position)
ITR = 80 #训练迭代次数
LR = 0.5 #学习速率
seed = 0 # Seed for random number generator
# q_delta = 0.001
# q_delta = paddle.to_tensor(q_delta,dtype = 'complex128')
#构造cz gate
a= np.array([np.pi/2])
cz_theta = paddle.to_tensor(a)
#coefficient of A=c_0 A_0 + c_1 A_1 ...
c = np.array([1,0.2,0.2])
c = paddle.to_tensor(c,dtype = 'complex128')


## 各门电路定义
U操作: $U|0\rangle = |b\rangle$  

A操作: $A = \mathbb{I}+0.2X_0Z_1+0.2X_0$

要训练的量子网络V $V|0\rangle = |x\rangle$

In [81]:
#定义门电路U U|0> = |b>
def U_b(cir):
    for idx in range(num_qubit):
        cir.h(idx)
# 定义受控酉矩阵
def CA(cir,idx):
    if idx == 0:
        # Identity operation
        None
    elif idx == 1:
        cir.cnot([ancilla_idx,0])
        #cz gate
        cir.s(ancilla_idx)
        cir.s(ancilla_idx)
        cir.s(ancilla_idx)
        cir.cnot([ancilla_idx,1])
        cir.rz(cz_theta,1)
        cir.cnot([ancilla_idx,1])
        cir.rz(-cz_theta,1)
    elif idx == 2:
        cir.cnot([ancilla_idx,0])
#定义变分量子电路 使得|x> = V|0>
def variational_block(cir,theta):
    '''
    QNN
    '''
    #第一层是给除附加位外所有量子比特施加h门
    for idx in range(num_qubit):
        cir.h(idx)
    #单层的全y门
    for idx, element in enumerate(theta):
        cir.ry(element,idx)


## hadamard test
根据hadamard test电路要求，按顺序放好构建的门序列。  

获取系统最终状态里附加位为$|0\rangle$的概率（由于paddle_quantum中没有可以直接测量系统中某条电路状态的方法，故根据测量原理，构建投影算子M，计算辅助位为$|0\rangle$概率）

根据hadamard test算法，只需要对辅助比特进行计算基测量，测量得到结果为$|0\rangle$的概率与两量子态内积有：$Re(\langle\phi_1|\phi\rangle = 2*p_0 -1$ 类似的，虚数部分需要添加相位变换s gate，相同的推导可以得到虚数部分$\operatorname{Im}\left(\left\langle\phi_{1} \mid \phi_{2}\right\rangle\right)=2 p_{0}-1$

In [82]:
#hadamard test
def local_hadamard_test(theta,l = None,lp=None,j = None,part = None):
    # print('test: l,lp,j', l,lp,j)
    #初始化电路
    cir = UAnsatz(tot_qubits)#需要附加位
    #作用在附加位上的H门
    cir.h(ancilla_idx)
    #如果要计算的是虚数部，附加位还需要加一个相位门
    #phase gate
    if part == 'Im' or part == 'im':
        cir.s(ancilla_idx)
    #生成一个向量｜x>
    variational_block(cir,theta)
    #使用A_l
    CA(cir,l)
    #添加U_b
    U_b(cir)
    #添加受控z
    if j != -1:
        #cz gate
        cir.s(ancilla_idx)
        cir.s(ancilla_idx)
        cir.s(ancilla_idx)
        cir.cnot([ancilla_idx,j])
        cir.rz(cz_theta,j)
        cir.cnot([ancilla_idx,j])
        cir.rz(-cz_theta,j)
    #添加U
    U_b(cir)
    #contralled gate为A的共轭转置
    CA(cir,lp)
    #辅助位第二个hadamard门
    cir.h(ancilla_idx)
    #对辅助比特为｜0>进行观测
    fi_state = cir.run_state_vector()
    state = paddle.reshape(fi_state, shape=(2**tot_qubits, 1))
    # print(state.shape)
    M_0 = np.array([[1,0],[0,0]])
    Id = np.identity(2)
    M = np.kron(np.kron(Id,Id),Id)
    M = np.kron(M,M_0)
    M = paddle.to_tensor(M)
    #计算辅助位｜0>的概率
    M = paddle.matmul(dagger(M), M)
    p0 = paddle.matmul(paddle.matmul(dagger(state), M), state) 
    b2t = paddle.to_tensor(2.0)
    p0 = paddle.matmul(b2t,p0)
    b1t = paddle.to_tensor(1.0)
    po = paddle.subtract(p0,b1t)
    # print(p0)
    return p0

## local cost function中重要系数$\mu$计算
根据论文中描述，可以知道局部cost function需使用hadamard test计算。$\delta_{l l^{\prime}}^{(j)}=\left\langle\mathbf{0}\left|V^{\dagger} A_{l^{\prime}}^{\dagger} U\left(\left|0_{j}\right\rangle\left\langle 0_{j}\right| \otimes \mathbb{I}_{\bar{J}}\right) U^{\dagger} A_{l} V\right| \mathbf{0}\right\rangle$，其中$\left|0_{j}\right\rangle\left\langle 0_{j}\right|=\left(\mathbb{I}_{j},Z_j)/2\right.$，所以有
$\delta_{l l^{\prime}}^{(j)}=\beta_{l l^{\prime}}+\left\langle\mathbf{0}\left|V^{\dagger} A_{l^{\prime}}^{\dagger} U\left(Z_{j} \otimes \mathbb{I}_{\bar{\jmath}}\right) U^{\dagger} A_{l} V\right| \mathbf{0}\right\rangle$

In [83]:
#计算local cost function的系数
def mu(theta,l = None,lp=None,j=None):
    # print('mu: l,lp,j= ',l,lp,j)
    mu_real = local_hadamard_test(theta,l = l,lp = lp,j = j,part='Re')
    mu_imag = local_hadamard_test(theta,l = l, lp = lp, j = j, part= 'Im')
    jj = paddle.to_tensor([1j],dtype = 'complex128')
    muval = mu_real+ mu_imag * jj
    return muval

## local cost function 计算
首先对$\langle x|A^{\dagger}A|x\rangle$的评估  
接着利用hadamard test算得的mu，求和计算loss值  
求loss值的过程中，三层循环相当于是对每个$A_l,A^{\dagger}_l,Z_j$做了一遍测量 即要做$3*3*3 = 27$次

In [84]:
# 归一化 先计算 <x|A'A|x> 分母
def psi_norm(theta):
    '''Returns the normalization constant <psi|psi>'''
    norm = 0
    norm = paddle.to_tensor(norm,dtype = 'complex128')
    for l in range(c.size):
        for lp in range(c.size):
            norm = norm+c[l]*paddle.conj(c[lp])*mu(theta,l,lp,-1)
    return paddle.abs(norm)
#计算cost function
def local_cost(theta):
    #当A｜x>和|b>同方向时，该值越接近0
    mu_sum = 0
    mu_sum = paddle.to_tensor(mu_sum,dtype = 'complex128')
    #对每一个A0-2
    for l in range(0, c.size):
        for lp in range(0, c.size):
            for j in range(0, num_qubit):
                mu_sum = mu_sum + c[l] * paddle.conj(c[lp]) * mu(theta, l, lp, j)
    
    mu_sum = paddle.abs(mu_sum)
    m1 = num_qubit * psi_norm(theta)
    result = 0.5 - 0.5 * mu_sum / m1
    # Cost function C_L
    return result


In [85]:
# theta = np.random.uniform(0,2*np.pi,num_qubit)
# theta = paddle.to_tensor(theta)
# # norm = psi_norm(theta)
# # print(norm)
# loss =local_cost(theta)
# print('%.10f' %loss)

## 优化--梯度下降
利用adam优化器，对角度参数$theta$进行优化，使得loss值不断减小，即$V|0\rangle - |b\rangle$趋于0  
要调用优化器时，涉及到的计算参数必须是paddle形式的计算 如果是numpy形式无法处理

In [86]:
class vqlsOpt(paddle.nn.Layer):
    def __init__(self, shape, dtype='float64'):
        super(vqlsOpt, self).__init__()
        # 初始化 theta 参数列表，并用 [0, 2*pi] 的均匀分布来填充初始值
        self.theta = self.create_parameter(shape=shape,default_initializer=paddle.nn.initializer.Uniform(low=0.0, high=2*np.pi), dtype=dtype, is_bias=False)
        
    # 定义损失函数和前向传播机制
    def forward(self):
        #初始化电路
        cir_v = UAnsatz(num_qubit)
        U = variational_block(cir_v,self.theta)
        #计算损失函数
        loss = local_cost(self.theta)
        return loss

## 迭代优化


In [87]:
# 定义网络维度
paddle.seed(seed)
vqls = vqlsOpt(shape = [3])
# 一般来说，我们利用Adam优化器来获得相对好的收敛，当然你可以改成SGD或者是RMS prop.
opt = paddle.optimizer.Adam(learning_rate = LR, parameters = vqls.parameters())    
# 优化循环
for itr in range(1,ITR+1):
    # 前向传播计算损失函数
    loss = vqls()
    # 反向传播极小化损失函数
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()
    print('iter:', itr, '  loss: %.7f' % loss.numpy())




iter: 1   loss: 0.0832684
iter: 2   loss: 0.0381729
iter: 3   loss: 0.0161175
iter: 4   loss: 0.0172738
iter: 5   loss: 0.0200259
iter: 6   loss: 0.0158604
iter: 7   loss: 0.0109704
iter: 8   loss: 0.0100892
iter: 9   loss: 0.0116899
iter: 10   loss: 0.0124964
iter: 11   loss: 0.0111305
iter: 12   loss: 0.0081296
iter: 13   loss: 0.0049659
iter: 14   loss: 0.0029259
iter: 15   loss: 0.0023992
iter: 16   loss: 0.0030672
iter: 17   loss: 0.0043916
iter: 18   loss: 0.0057376
iter: 19   loss: 0.0063944
iter: 20   loss: 0.0058587
iter: 21   loss: 0.0041897
iter: 22   loss: 0.0020685
iter: 23   loss: 0.0004625
iter: 24   loss: 0.0000577
iter: 25   loss: 0.0008150
iter: 26   loss: 0.0020157
iter: 27   loss: 0.0027936
iter: 28   loss: 0.0027017
iter: 29   loss: 0.0019028
iter: 30   loss: 0.0009476
iter: 31   loss: 0.0003611
iter: 32   loss: 0.0003199
iter: 33   loss: 0.0006217
iter: 34   loss: 0.0009214
iter: 35   loss: 0.0010023
iter: 36   loss: 0.0008727
iter: 37   loss: 0.0006715
iter: 38  

In [88]:
print(vqls.theta)

Parameter containing:
Tensor(shape=[3], dtype=float64, place=CPUPlace, stop_gradient=False,
       [0.00705842, 0.32586962, 6.28196748])


In [92]:
#验证
weight = vqls.theta
cir_test = UAnsatz(num_qubit)
def prepare_x(theta):
    a = variational_block(cir_test,theta)
    # U= cir_test.U.numpy()
    # print(U)
    x_0 = cir_test.run_state_vector()
    print(x_0.numpy())
prepare_x(weight)
    



[-0.29066532+0.j -0.29031155+0.j -0.40503258+0.j -0.40453962+0.j
 -0.29272423+0.j -0.29236796+0.j -0.40790161+0.j -0.40740516+0.j]


## 量子电路结果和经典结果比较
获得优化器中最终的$theta$参数列表，重新初始化量子门电路$V$，带入参数$theta$，观察计算结果

## 传统numpy方法计算
$Ax=b$ 计算得到x的真实值 用于和量子方法进行对比

In [15]:
Id = np.identity(2)
Z = np.array([[1, 0], [0, -1]])
X = np.array([[0, 1], [1, 0]])
ct = [1,0.2,0.2]
A_0 = np.identity(8)
A_1 = np.kron(np.kron(X, Z), Id)
A_2 = np.kron(np.kron(X, Id), Id)

A_num = ct[0] * A_0 + ct[1] * A_1 + ct[2] * A_2
b = np.ones(8) / np.sqrt(8)
print("A = \n", A_num)
print("b = \n", b)
A_inv = np.linalg.inv(A_num)
x_real = np.dot(A_inv, b)
print('x = \n',x_real)


A = 
 [[1.  0.  0.  0.  0.4 0.  0.  0. ]
 [0.  1.  0.  0.  0.  0.4 0.  0. ]
 [0.  0.  1.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  1.  0.  0.  0.  0. ]
 [0.4 0.  0.  0.  1.  0.  0.  0. ]
 [0.  0.4 0.  0.  0.  1.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  1.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  1. ]]
b = 
 [0.35355339 0.35355339 0.35355339 0.35355339 0.35355339 0.35355339
 0.35355339 0.35355339]
x = 
 [0.25253814 0.25253814 0.35355339 0.35355339 0.25253814 0.25253814
 0.35355339 0.35355339]
/usr/local/anaconda3/envs/paddle_quantum_env/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [756]:
a = [5.0,8.3,2.5]

b = paddle.to_tensor(a)
jj = paddle.to_tensor([1j])
print(b.size)
print(jj)
print(jj.size)

3
Tensor(shape=[1], dtype=complex64, place=CPUPlace, stop_gradient=True,
       [1j])
1
/usr/local/anaconda3/envs/paddle_quantum_env/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [804]:
a1 = np.array([[1,0],[0,0]])
a2 = np.identity(2)
p = np.kron(a1,a2)
p = paddle.to_tensor(p)
#计算辅助位｜0>的概率
p = paddle.matmul(dagger(p), p)
print(p)
a = np.array([1, 2, 3,4])
b = a.reshape(-1, 1)
a1=paddle.to_tensor(a,dtype = 'float64')
print(a.shape)
b1 = paddle.to_tensor(b,dtype = 'float64')
print(b.shape)
a = b.reshape(1,4)
a1=paddle.to_tensor(a,dtype = 'float64')
print(a.shape)
iouy = paddle.matmul(p,dagger(p))
iou = paddle.matmul(paddle.matmul(a1,iouy),b1)
print(iou)

print(iou*4)


Tensor(shape=[4, 4], dtype=float64, place=CPUPlace, stop_gradient=True,
       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
(4,)
(4, 1)
(1, 4)
Tensor(shape=[1, 1], dtype=float64, place=CPUPlace, stop_gradient=True,
       [[5.]])
Tensor(shape=[1, 1], dtype=float64, place=CPUPlace, stop_gradient=True,
       [[20.]])


In [840]:
ooo = 2 
oo = paddle.to_tensor(ooo,dtype = 'complex128')
print(oo)
jj = paddle.to_tensor([1j],dtype = 'complex128')
print(jj)
print(oo*jj)

Tensor(shape=[1], dtype=complex128, place=CPUPlace, stop_gradient=True,
       [(2+0j)])
Tensor(shape=[1], dtype=complex128, place=CPUPlace, stop_gradient=True,
       [1j])
Tensor(shape=[1], dtype=complex128, place=CPUPlace, stop_gradient=True,
       [2j])
/usr/local/anaconda3/envs/paddle_quantum_env/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
